In [1]:
import Acqu as aq

# Acqu Module
The Acqu module parses in Mk1 and Mk2 data files which can be read event by event. A numpy array is made of form, (ADC,Value) for each event stored as **adcArray**. If the event contains scaler or epics data this can also be read but hasn't been tested recently.


inFile = '/scratch/Timepix_33.dat'
aq.openFile(inFile)

# Define functions
The module allows any arbitrary function to be run on each event. This is passed as an argument to the **runFunction** function in the Acqu module. The other arguments are the number of the first and last event to be read.

Here the function just prints the event number and the raw adcArray:


def printRAW():
    print('number',aq.eventNo)
    print(aq.adcArray)

aq.runFunction(printRAW,0,10)

import ROOT
import numpy as np
import root_numpy
from rootpy.plotting import histogram, Hist2D, Hist, Canvas

adchist     = Hist(10000,0,10000,name='adcs')
adchist2d   = Hist2D(390,800,1190,68000,0,68000,name='adcvalues')

def fillADCHist():
    adchist.fill_array(aq.adcArray['adc'])
    adchist2d.fill_array(np.vstack((aq.adcArray['adc'],aq.adcArray['val'])).T)    
    if(aq.eventNo%10000==0):
        print('number',aq.eventNo)
        
aq.runFunction(fillADCHist,0,100000)     

ROOT.enableJSVis()
c1 = ROOT.TCanvas("my1","The Canvas Title",1000,600)
c1.Divide(2,1)
c1.cd(1)
adchist.Draw("colz")
c1.cd(2)
adchist2d.Draw("colz")
c1.Draw()

In [2]:
import AcquDetector as aqdet

#ADC 104 against PPOL Event
import Timepix


ADC104 = 0.0
def plotTimepix():
    global ADC104
        
    if(aq.epicsEvent==1):
        print len(aq.adcArray)
        #print aq.adcArray
        eventA       = aq.getEpicsPV('PPOL:TIMEPIXA:EVENT')
        eventB       = aq.getEpicsPV('PPOL:TIMEPIXB:EVENT')
        print eventA, eventB
        print ADC104
    else:        
        ADC104 = aq.adcArray[np.where(aq.adcArray['adc']==104)[0]]['val'][0]
        
aq.runFunction(plotTimepix,0,5000)

import Timepix

tpxXY1      = Hist2D(256,0,256,256,0,256)
tpxXY2      = Hist2D(256,0,256,256,0,256)
tpxTime     = Hist(10000,0,1)

def plotTimepix():
    if(aq.epicsEvent==1):
        nHitsA       = aq.getEpicsPV('PPOL:TIMEPIXA:NHITS')
        encodedA     = aq.getEpicsPV('PPOL:TIMEPIXA:ENCODED')
        nHitsB       = aq.getEpicsPV('PPOL:TIMEPIXB:NHITS')
        encodedB     = aq.getEpicsPV('PPOL:TIMEPIXB:ENCODED')
        TimepixAData = Timepix.Decode(nHitsA,encodedA)
        TimepixBData = Timepix.Decode(nHitsB,encodedB)
        tpxXY1.fill_array(np.vstack((TimepixAData['y'],TimepixAData['x'])).T)
        tpxXY2.fill_array(np.vstack((TimepixBData['y'],TimepixBData['x'])).T)
        nsTimeA = 25*TimepixAData[['ToA']].astype(float) - 25/16*TimepixAData[['FToA']].astype(float)
        tpxTime.fill_array(nsTimeA/1000000000)
       
aq.runFunction(plotTimepix,0,50000)

ROOT.enableJSVis()
c4 = ROOT.TCanvas("my4","The Canvas Title",1000,600)
c4.Divide(2,1)
c4.cd(1)
tpxXY1.Draw("colz")
c4.cd(2)
tpxXY2.Draw("colz")
#c4.cd(3)
#tpxTime.Draw("colz")
c4.Draw()

In [3]:
import Timepix
import ROOT
import numpy as np
import root_numpy
from rootpy.plotting import histogram, Hist2D, Hist, Canvas
inFile = '/scratch/2019-05_Timepix/Timepix_33.dat'
aq.openFile(inFile)

triggerTime  = []
scalerRepeat = 0
prevLong     = 0
pulseCount   = 0
EPICSN       = 0
EPICSEventA  = 0
EPICSEventB  = 0

def TestTimings():
    global prevTriggerTime
    global triggerTime
    global scalerRepeat
    global prevLong
    global pulseCount
    global EPICSEventA
    global EPICSEventB
    global EPICSN
    
    
    ADC104    = aq.adcArray[np.where(aq.adcArray['adc']==104)[0]]['val'][0]
    #shortTime = aq.adcArray[np.where(aq.adcArray['adc']==102)[0]]['val'][0]
    #longTime  = aq.adcArray[np.where(aq.adcArray['adc']==103)[0]]['val'][0]
    #if(longTime<prevLong): scalerRepeat += 1
    #triggerTime += [8*(shortTime+65536*(longTime+65536*scalerRepeat))]
    #if(len(triggerTime)<=1): return
    if(aq.epicsEvent==1):
        if(EPICSEventA!=aq.getEpicsPV('PPOL:TIMEPIXA:EVENT') and EPICSEventB!=aq.getEpicsPV('PPOL:TIMEPIXB:EVENT') ):
            print aq.getEpicsPV('PPOL:TIMEPIXA:EVENT'), aq.getEpicsPV('PPOL:TIMEPIXB:EVENT'), pulseCount
            EPICSEventA  = aq.getEpicsPV('PPOL:TIMEPIXA:EVENT')
            EPICSEventB  = aq.getEpicsPV('PPOL:TIMEPIXB:EVENT')
          
        
    if(aq.eventNo%10000==0):
        print("Events Processed: ",aq.eventNo)
    if(ADC104!=pulseCount):
        pulseCount  = ADC104
        #prevTriggerTime = triggerTime[-1]
        triggerTime = []
    
aq.runFunction(TestTimings,0,50000)



Welcome to JupyROOT 6.16/00
Mk2 Data
83.0 83.0 0
84.0 84.0 516
85.0 85.0 517
86.0 86.0 520
87.0 87.0 521
88.0 88.0 524
89.0 89.0 525
('Events Processed: ', 10000)
90.0 90.0 528
91.0 91.0 529
92.0 92.0 532
93.0 93.0 533
94.0 94.0 536
95.0 95.0 537
('Events Processed: ', 20000)
96.0 96.0 540
97.0 97.0 541
98.0 98.0 544
99.0 99.0 545
100.0 100.0 548
101.0 101.0 549
102.0 102.0 552
('Events Processed: ', 30000)
103.0 103.0 553
104.0 104.0 556
105.0 105.0 557
106.0 106.0 560
107.0 107.0 561
108.0 108.0 564
109.0 109.0 565
('Events Processed: ', 40000)
110.0 110.0 567
111.0 111.0 569
112.0 112.0 571
113.0 113.0 574
114.0 114.0 575
115.0 115.0 578
('Events Processed: ', 50000)


In [4]:
import Timepix
import ROOT
import numpy as np
import root_numpy
from rootpy.plotting import histogram, Hist2D, Hist, Canvas
inFile = '/scratch/2019-05_Timepix/Timepix_33.dat'
aq.openFile(inFile)

triggerTime  = []
scalerRepeat = 0
prevLong     = 0
pulseCount   = 0
EPICSN       = 0
EPICSEventA  = 0
EPICSEventB  = 0

timeWindowMin = -400000
timeWindowMax = 400000

timeMax = 200000000
nBins = 300


timeWin       = 2000000
timeDiffDouble      = Hist(nBins,-timeWin,timeWin,name='DoubleCoincidence')
timeDiff      = Hist(nBins,-timeWin,timeWin,name='Coincidence')

timeA         = Hist(40000,0,timeMax,name='TimeA')
timeB         = Hist(40000,0,timeMax,name='TimeB')

#h = Hist2D((timeWindowMax-timeWindowMin)/1000,timeWindowMin,timeWindowMax,40,0,40)
hA  = Hist((timeWindowMax-timeWindowMin)/5000,timeWindowMin,timeWindowMax,name='TimesA')
hA2  = Hist((timeWindowMax-timeWindowMin)/5000,timeWindowMin,timeWindowMax,name='TimesA2')
h2A = Hist2D((timeWindowMax-timeWindowMin)/5000,timeWindowMin,timeWindowMax,100,0,timeMax,name='TimesA2D')
hB  = Hist((timeWindowMax-timeWindowMin)/5000,timeWindowMin,timeWindowMax,name='TimesB')
h2B = Hist2D((timeWindowMax-timeWindowMin)/5000,timeWindowMin,timeWindowMax,100,0,timeMax,name='TimesB2D')
prevTriggerTime = [0]

positionA = Hist2D(256,0,256,256,0,256,name='posA')
positionACoin = Hist2D(256,0,256,256,0,256,name='posACoin')

def TimepixTagger():
    global prevTriggerTime
    global triggerTime
    global scalerRepeat
    global prevLong
    global pulseCount
    global EPICSEventA
    global EPICSEventB
    global EPICSN
    
    
    ADC104    = aq.adcArray[np.where(aq.adcArray['adc']==104)[0]]['val'][0]
    shortTime = aq.adcArray[np.where(aq.adcArray['adc']==102)[0]]['val'][0]
    longTime  = aq.adcArray[np.where(aq.adcArray['adc']==103)[0]]['val'][0]
    if(longTime<prevLong): scalerRepeat += 1
    triggerTime += [8*(shortTime+65536*(longTime+65536*scalerRepeat))]
    
    if(len(triggerTime)<=1): return
    
    if(aq.epicsEvent==1):
        if(EPICSEventA!=aq.getEpicsPV('PPOL:TIMEPIXA:EVENT') and EPICSEventB!=aq.getEpicsPV('PPOL:TIMEPIXB:EVENT') ):
            EPICSEventA  = aq.getEpicsPV('PPOL:TIMEPIXA:EVENT')
            EPICSEventB  = aq.getEpicsPV('PPOL:TIMEPIXB:EVENT')
            nHitsA       = aq.getEpicsPV('PPOL:TIMEPIXA:NHITS')
            encodedA     = aq.getEpicsPV('PPOL:TIMEPIXA:ENCODED')
            nHitsB       = aq.getEpicsPV('PPOL:TIMEPIXB:NHITS')
            encodedB     = aq.getEpicsPV('PPOL:TIMEPIXB:ENCODED')
            TimepixAData = Timepix.Decode(nHitsA,encodedA)
            TimepixBData = Timepix.Decode(nHitsB,encodedB)
            nsTimeA = 16*TimepixAData[['ToA']].astype(long) - TimepixAData[['FToA']].astype(long)
            nsTimeB = 16*TimepixBData[['ToA']].astype(long) - TimepixBData[['FToA']].astype(long)
            nsTimeA.sort()
            nsTimeB.sort()
            diffA = np.concatenate([[0],np.diff(nsTimeA)])
            diffB = np.concatenate([[0],np.diff(nsTimeB)])            
            breakA = np.concatenate([[0],np.where(diffA>200)[0],[len(nsTimeA)]])
            breakB = np.concatenate([[0],np.where(diffB>200)[0],[len(nsTimeB)]])
            timesA = []
            timesB = []
            lengA  = []
            lengB  = []
            posA   = []
            posB   = []
            if(len(nsTimeA)==0): breakA = []
            if(len(nsTimeB)==0): breakB = []
            for ssi in zip(breakA[:-1],breakA[1:]):
                diffs = nsTimeA[ssi[0]:ssi[1]]-nsTimeA[ssi[0]]
                leng  = ssi[1]-ssi[0]
                comb  = np.vstack([diffs,np.full(leng,leng)]).T
                lengA  += [leng]
                timesA += [nsTimeA[ssi[0]]]
                posA   += [TimepixAData[['x','y']][ssi[0]]]
            for ssi in zip(breakB[:-1],breakB[1:]):
                diffs = nsTimeB[ssi[0]:ssi[1]]-nsTimeB[ssi[0]]
                leng  = ssi[1]-ssi[0]
                comb  = np.vstack([diffs,np.full(leng,leng)]).T
                lengB  += [leng]
                timesB += [nsTimeB[ssi[0]]]
                posB   += [TimepixBData[['x','y']][ssi[0]]]
            
            timesA = np.array(timesA)
            timesB = np.array(timesB)
            posA = np.array(posA)
            posB = np.array(posB)
            
            
            positionA.fill_array(np.vstack([posA['x'],posA['y']]).T)
            timeA.fill_array(timesA)
            timeB.fill_array(timesB)
                        
            EPICSN += 1
            triggerTime2 = np.array(triggerTime,dtype=np.float)
            triggerWrap  = np.where(np.diff(triggerTime2)<0)[0]
            if(len(triggerWrap)>0):
                triggerTime2[triggerWrap[0]:] += 65536*65536*8
                
            timeSub = prevTriggerTime[-1]
            #print timeSub
            if((triggerTime2-timeSub)[-1]<1000000000 and len(prevTriggerTime)>1): timeSub = prevTriggerTime[-2]
            #print timeSub
            triggerTime2 -= timeSub#triggerTime[0]
            triggerTime2 /= 1.015595*1.5625
            #triggerTime2 /= 1.5625
            
            #print triggerTime2
            #print timesA
            
            coinIdx = []
            
            for i, hit in enumerate(timesA):
                #print -nsTimeA+hit
                tdiff = triggerTime2-hit
                #print tdiff
                #h2A.fill_array(np.vstack((tdiff,np.full(len(triggerTime2),hit))).T)
                #hA.fill_array(tdiff)
                #hA2.fill_array(tdiff-263000)
                if (len(tdiff[np.abs(tdiff)<10000])>0 or len(tdiff[np.abs(tdiff-263000)<10000])>0):
                    coinIdx +=  [i]
            #print len(coinIdx), len(timesA)
            #print triggerTime2[0], triggerTime2[-1], prevTriggerTime
            #if(len(coinIdx)<80): return
            
            for i, hit in enumerate(timesA):
                tdiff = triggerTime2-hit
                #print tdiff
                h2A.fill_array(np.vstack((tdiff,np.full(len(triggerTime2),hit))).T)
                hA.fill_array(tdiff)
                hA2.fill_array(tdiff-263000)
                
                
            coinHit  = timesA[coinIdx]
            coinPosA = posA[coinIdx]
            positionACoin.fill_array(np.vstack([coinPosA['x'],coinPosA['y']]).T)
            
            
            
            for hit in timesB:
                tdiff = triggerTime2-hit
                #print -nsTimeA+hit
                h2B.fill_array(np.vstack((tdiff,np.full(len(triggerTime2),hit))).T)
                hB.fill_array(tdiff)
                tpxDiff  = coinHit-hit
                tpxDiff2 = timesA-hit
                timeDiff.fill_array(tpxDiff2)
                timeDiffDouble.fill_array(tpxDiff)
            #for hit in triggerTime:
                #print -nsTimeA+hit
                #h2.fill_array(np.vstack((nsTimeA-hit,np.full(len(nsTimeA),hit))).T)
                #h.fill_array(nsTimeA-hit)
                
            triggerfilter = prevTriggerTime[-1]
            if(triggerTime[0]>triggerfilter): triggerfilter += 65536*65536*8
            triggerTime     = triggerTime[np.where(triggerTime<=triggerfilter)[0][-1]:]
                
            prevTriggerTime = []
            
            
    if(aq.eventNo%10000==0):
        print("Events Processed: ",aq.eventNo)
    if(ADC104!=pulseCount):
        #print pulseCount, ADC104
        pulseCount  = ADC104        
        #print len(triggerTime)
        #print triggerTime[-1]-triggerTime[0]
        #print ' '
        prevTriggerTime += [triggerTime[-1]]
        #triggerTime = []
    
aq.runFunction(TimepixTagger,0,5000000)



Mk2 Data
('Events Processed: ', 10000)
('Events Processed: ', 20000)
('Events Processed: ', 30000)
('Events Processed: ', 40000)
('Events Processed: ', 50000)
('Events Processed: ', 60000)
('Events Processed: ', 70000)
('Events Processed: ', 80000)
('Events Processed: ', 90000)
('Events Processed: ', 100000)
('Events Processed: ', 110000)
('Events Processed: ', 120000)
('Events Processed: ', 130000)
('Events Processed: ', 140000)
('Events Processed: ', 150000)
('Events Processed: ', 160000)
('Events Processed: ', 170000)
('Events Processed: ', 180000)
('Events Processed: ', 190000)
('Events Processed: ', 200000)
('Events Processed: ', 210000)
('Events Processed: ', 220000)
('Events Processed: ', 230000)
('Events Processed: ', 240000)
('Events Processed: ', 250000)
('Events Processed: ', 260000)
('Events Processed: ', 270000)
('Events Processed: ', 280000)
('Events Processed: ', 290000)
('Events Processed: ', 300000)
('Events Processed: ', 310000)
('Events Processed: ', 320000)
('Events

KeyboardInterrupt: 

In [13]:

ROOT.enableJSVis()
c41 = ROOT.TCanvas("my41","The Canvas Title",1000,600)
positionA.Draw("colz")
positionA.GetXaxis().SetTitle("y position?")
positionA.GetYaxis().SetTitle("x position?")
c41.Draw()
c42 = ROOT.TCanvas("my42","The Canvas Title",1000,600)
positionACoin.Draw("colz")
positionACoin.GetXaxis().SetTitle("y position at the coincidence?")
positionACoin.GetYaxis().SetTitle("x position at the coincidence?")
c42.Draw()


ROOT.enableJSVis()
c51 = ROOT.TCanvas("my51","The Canvas Title",1000,600)
positionA.RebinX(2)
positionA.RebinY(2)
positionA.Draw("colz")
c51.Draw()
c52 = ROOT.TCanvas("my52","The Canvas Title",1000,600)
positionACoin.RebinX(2)
positionACoin.RebinY(2)
positionACoin.Draw("colz")
c52.Draw()

In [14]:

ROOT.enableJSVis()
c3 = ROOT.TCanvas("my3","The Canvas Title",1000,600)
timeDiff.Draw("colz")
timeDiff.GetXaxis().SetTitle("time diff between A and B")
c3.Draw()
c31 = ROOT.TCanvas("my31","The Canvas Title",1000,600)
timeDiffDouble.Draw("colz")
timeDiffDouble.GetXaxis().SetTitle("time diff between coincidence at A and time at B")
c31.Draw()

ROOT.enableJSVis()
c21 = ROOT.TCanvas("my21","The Canvas Title",1200,600)
timeA.Draw("colz")
c21.Draw()
c22 = ROOT.TCanvas("my22","The Canvas Title",1200,600)
timeB.Draw("colz")
c22.Draw()

In [5]:
ROOT.enableJSVis()
c51 = ROOT.TCanvas("my51","The Canvas Title",1000,600)
hA.Draw("colz")
hA.GetXaxis().SetTitle("time diff between trigger time and time at A")
c51.Draw()
c50 = ROOT.TCanvas("my50","The Canvas Title",1000,600)
hA2.Draw("colz")
hA2.GetXaxis().SetTitle("time diff between trigger time and time at A, shifted")
c50.Draw()
c52 = ROOT.TCanvas("my52","The Canvas Title",1000,600)
h2A.Draw("colz")
h2A.GetXaxis().SetTitle("time diff between trigger time and time at A???")
h2A.GetYaxis().SetTitle("B time???")
h2A.GetZaxis().SetTitle("freq??")
c52.Draw()
c53 = ROOT.TCanvas("my53","The Canvas Title",1000,600)
hB.Draw("colz")
hB.GetXaxis().SetTitle("time diff between trigger time and time at B")
c53.Draw()
c54 = ROOT.TCanvas("my54","The Canvas Title",1000,600)
h2B.Draw("colz")
h2B.GetXaxis().SetTitle("???")
c54.Draw()